Public Github: https://github.com/kchand12/DL4H_Team_149/tree/main

Author: Kanjana Chandren (kanjana3@illinois.edu)

# Before you use this template

This template is just a recommended template for project Report. It only considers the general type of research in our paper pool. Feel free to edit it to better fit your project. You will iteratively update the same notebook submission for your draft and the final submission. Please check the project rubriks to get a sense of what is expected in the template.

---

# FAQ and Attentions
* Copy and move this template to your Google Drive. Name your notebook by your team ID (upper-left corner). Don't eidt this original file.
* This template covers most questions we want to ask about your reproduction experiment. You don't need to exactly follow the template, however, you should address the questions. Please feel free to customize your report accordingly.
* any report must have run-able codes and necessary annotations (in text and code comments).
* The notebook is like a demo and only uses small-size data (a subset of original data or processed data), the entire runtime of the notebook including data reading, data process, model training, printing, figure plotting, etc,
must be within 8 min, otherwise, you may get penalty on the grade.
  * If the raw dataset is too large to be loaded  you can select a subset of data and pre-process the data, then, upload the subset or processed data to Google Drive and load them in this notebook.
  * If the whole training is too long to run, you can only set the number of training epoch to a small number, e.g., 3, just show that the training is runable.
  * For results model validation, you can train the model outside this notebook in advance, then, load pretrained model and use it for validation (display the figures, print the metrics).
* The post-process is important! For post-process of the results,please use plots/figures. The code to summarize results and plot figures may be tedious, however, it won't be waste of time since these figures can be used for presentation. While plotting in code, the figures should have titles or captions if necessary (e.g., title your figure with "Figure 1. xxxx")
* There is not page limit to your notebook report, you can also use separate notebooks for the report, just make sure your grader can access and run/test them.
* If you use outside resources, please refer them (in any formats). Include the links to the resources if necessary.

# Mount Notebook to Google Drive
Upload the data, pretrianed model, figures, etc to your Google Drive, then mount this notebook to Google Drive. After that, you can access the resources freely.

Instruction: https://colab.research.google.com/notebooks/io.ipynb

Example: https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q

Video: https://www.youtube.com/watch?v=zc8g8lGcwQU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Introduction
This is an introduction to your report, you should edit this text/mardown section to compose. In this text/markdown, you should introduce:

*   Background of the problem
  * The paper focuses on predictive modeling using deep learning to predicut ICU length of stay (LoS). The problem involves feature engineering and data processing to address issues in EHR data.
  * Accurate prediction of ICU LoS is important for hospital resource management to estimate staffing, bed allocation and resource optimization for better patient care quality.
  * The problem is challenging due to the nature of ICU data which is sparse and irregularly sampled. The variability in patient conditions and treamtment responses makes standardization and accuract prediction hard.
  * Prior methods include Long Short-Term Memory(LSTM) networks however there were challenges in handling ICU data specifically the models need for real-time updates and managing data
*   Paper explanation
  * The paper proposes a new model, the Temporal Pointwise Convolution (TPC) network, that addresses the challenges of ICU LoS prediction more effectively than existing methods.
  * The TPC model combines temporal convolutional layers to capture time-dependent patterns and pointwise convolutional layers to extract features from complex interactions in patient data. This combination is works well at processing irregular and skewed data typical of ICU records.
  * The Temporal Pointwise Convolution (TPC) model significantly outperformed other baseline models, achieving a Mean Absolute Deviation (MAD) of 1.55 days on the eICU dataset and 2.28 days on the MIMIC-IV dataset, and reducing the Mean Squared Logarithmic Error (MSLE) to 0.70 and 0.39, respectively.
  * The introduction of the TPC model represents a significant advancement in predictive modeling for ICU management. It addresses specific data challenges in an ICU setting and outperforms existing models. This work contributes to both the theoretical and practical aspects of patient care optimization.

  




In [ ]:
# code comment is used as inline annotations for your coding

# Scope of Reproducibility:

List hypotheses from the paper you will test and the corresponding experiments you will run.


1.   Hypothesis 1: The TPC model will outperform traditional deep learning models like Long Short-Term Memory and Transformer models in accurately predicting the remaining length of stay for patients in the ICU.

You can insert images in this notebook text, [see this link](https://stackoverflow.com/questions/50670920/how-to-insert-an-inline-image-in-google-colaboratory-from-google-drive) and example below:

![sample_image.png](https://drive.google.com/uc?export=view&id=1g2efvsRJDxTxKz-OY3loMhihrEUdBxbc)



You can also use code to display images, see the code below.

The images must be saved in Google Drive first.


In [ ]:
# no code is required for this section
'''
if you want to use an image outside this notebook for explanaition,
you can upload it to your google drive and show it with OpenCV or matplotlib
'''
# mount this notebook to your google drive
drive.mount('/content/gdrive')

# define dirs to workspace and data
img_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-your-image>'

import cv2
img = cv2.imread(img_dir)
cv2.imshow("Title", img)


# Methodology

This methodology is the core of your project. It consists of run-able codes with necessary annotations to show the expeiment you executed for testing the hypotheses.

The methodology at least contains two subsections **data** and **model** in your experiment.

In [ ]:
# import  packages you need
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import argspace
import json
import os
from itertools import islice
from google.colab import drive


##  Data
Data includes raw data (MIMIC III tables), descriptive statistics (our homework questions), and data processing (feature engineering).
  * Source of the data: The data is from the eICU Collaborative Research Database, which is an intensive care unit database available by Philips Healthcare in partnership with the MIT Laboratory for Computational Physiology. This dataset includes a variety of de-identified data from over 200,000 ICU admissions across the United States. The dataset can be accessed from PhysioNet, and accessing this data requires approval after agreeing to a specified use agreement.
  * Statistics: For modeling purposes, the dataset is split 70% for training, 15% for validation, and 15% for testing.
  * Data process: Data from various tables like patient, lab, nurseCharting and processed to create feature. The data is organized into:
    * Time-series data that continously measure variables like heart rate and blood pressure.
    * Diagnoses data from diagnostic codes during ICU stay.
    * Flat features which are static information about patient demographics and detsils.
  Labels are created for clinical outcomes like mortality, duration of stay and remission. The data is also split into training, validation and testing so it can generalize well. Lastly all continous variables are normazlied for model training.
  * Illustration: printing results, plotting figures for illustration.
  * You can upload your raw dataset to Google Drive and mount this Colab to the same directory. If your raw dataset is too large, you can upload the processed dataset and have a code to load the processed dataset.

In [ ]:
# dir and function to load raw data
raw_data_dir = '/content/gdrive/My Drive/Colab Notebooks/eICU_data'

def load_raw_data(raw_data_dir):
  # implement this function to load raw data to dataframe/numpy array/tensor
  paths = {
        'diagnoses': f'{raw_data_dir}/diagnoses.csv',
        'flat_features': f'{raw_data_dir}/flat_features.csv',
        'labels': f'{raw_data_dir}/labels.csv',
        'timeseries_aperiodic': f'{raw_data_dir}/timeseriesaperiodic.csv',
        'timeseries_lab': f'{raw_data_dir}/timeserieslab.csv',
        'timeseries_nurse': f'{raw_data_dir}/timeseriesnurse.csv',
        'timeseries_periodic': f'{raw_data_dir}/timeseriesperiodic.csv',
        'timeseries_resp': f'{raw_data_dir}/timeseriesresp.csv'
    }

  data = {key: pd.read_csv(path) for key, path in paths.items()}

  return data

raw_data = load_raw_data(raw_data_dir)


In [ ]:
#This section is processing the timeseries data
def reconfigure_timeseries(timeseries, offset_column, feature_column=None, test=False):
    if test:
        timeseries = timeseries.iloc[300000:5000000]
    timeseries.set_index(['patientunitstayid', pd.to_timedelta(timeseries[offset_column], unit='T')], inplace=True)
    timeseries.drop(columns=offset_column, inplace=True)
    if feature_column is not None:
        timeseries = timeseries.pivot_table(columns=feature_column, index=timeseries.index)
    timeseries.index = pd.MultiIndex.from_tuples(timeseries.index, names=['patient', 'time'])
    return timeseries

def resample_and_mask(timeseries, eICU_path, header, mask_decay=True, decay_rate=4/3, test=False,
                       verbose=False, length_limit=24*14):
    if test:
        mask_decay = False
        verbose = True
    if verbose:
        print('Resampling to 1 hour intervals...')
    # take the mean of any duplicate index entries for unstacking
    timeseries = timeseries.groupby(level=[0, 1]).mean()

    timeseries.reset_index(level=1, inplace=True)
    timeseries.time = timeseries.time.dt.ceil(freq='H')
    timeseries.set_index('time', append=True, inplace=True)
    timeseries.reset_index(level=0, inplace=True)
    resampled = timeseries.groupby('patient').resample('H', closed='right', label='right').mean().drop(columns='patient')
    del (timeseries)

    def apply_mask_decay(mask_bool):
        mask = mask_bool.astype(int)
        mask.replace({0: np.nan}, inplace=True)  # so that forward fill works
        inv_mask_bool = ~mask_bool
        count_non_measurements = inv_mask_bool.cumsum() - \
                                 inv_mask_bool.cumsum().where(mask_bool).ffill().fillna(0)
        decay_mask = mask.ffill().fillna(0) / (count_non_measurements * decay_rate).replace(0, 1)
        return decay_mask

    # store which values had to be imputed
    if mask_decay:
        if verbose:
            print('Calculating mask decay features...')
        mask_bool = resampled.notnull()
        mask = mask_bool.groupby('patient').transform(apply_mask_decay)
        del (mask_bool)
    else:
        if verbose:
            print('Calculating binary mask features...')
        mask = resampled.notnull()
        mask = mask.astype(int)

    if verbose:
        print('Filling missing data forwards...')
    # carry forward missing values (note they will still be 0 in the nulls table)
    resampled = resampled.fillna(method='ffill')

    # simplify the indexes of both tables
    mask = mask.rename(index=dict(zip(mask.index.levels[1],
                                      mask.index.levels[1].days*24 + mask.index.levels[1].seconds//3600)))
    resampled = resampled.rename(index=dict(zip(resampled.index.levels[1],
                                                resampled.index.levels[1].days*24 +
                                                resampled.index.levels[1].seconds//3600)))

    # clip to length_limit
    if length_limit is not None:
        within_length_limit = resampled.index.get_level_values(1) < length_limit
        resampled = resampled.loc[within_length_limit]
        mask = mask.loc[within_length_limit]

    if verbose:
        print('Filling in remaining values with zeros...')
    resampled.fillna(0, inplace=True)

    # rename the columns in pandas for the mask so it doesn't complain
    mask.columns = [str(col) + '_mask' for col in mask.columns]

    # merge the mask with the features
    final = pd.concat([resampled, mask], axis=1)
    final.reset_index(level=1, inplace=True)
    final = final.loc[final.time > 0]

    if verbose:
        print('Saving progress...')
    # save to csv
    if test is False:
        final.to_csv(eICU_path + 'preprocessed_timeseries.csv', mode='a', header=header)
    return

def gen_patient_chunk(patients, size=1000):
    it = iter(patients)
    chunk = list(islice(it, size))
    while chunk:
        yield chunk
        chunk = list(islice(it, size))

def gen_timeseries_file(eICU_path, test=False):
    print('==> Loading data from timeseries files...')
    if test:
        # Load a subset of each file for testing purposes
        timeseries_lab = raw_data['timeseries_lab'].iloc[:500000]
        timeseries_resp = raw_data['timeseries_resp'].iloc[:500000]
        timeseries_nurse = raw_data['timeseries_nurse'].iloc[:500000]
        timeseries_periodic = raw_data['timeseries_periodic'].iloc[:500000]
        timeseries_aperiodic = raw_data['timeseries_aperiodic'].iloc[:500000]
    else:
        timeseries_lab = raw_data['timeseries_lab']
        timeseries_resp = raw_data['timeseries_resp']
        timeseries_nurse = raw_data['timeseries_nurse']
        timeseries_periodic = raw_data['timeseries_periodic']
        timeseries_aperiodic = raw_data['timeseries_aperiodic']

    # Continue with data reconfiguration and further processing...
    print('==> Reconfiguring lab timeseries...')
    timeseries_lab = reconfigure_timeseries(timeseries_lab, offset_column='labresultoffset', feature_column='labname', test=test)
    timeseries_lab.columns = [col.split('_')[1] if '_' in col else col for col in timeseries_lab.columns]  # Simplifying column names if necessary
    print('==> Reconfiguring respiratory timeseries...')
    # get rid of % signs (found in FiO2 section) and then convert into numbers
    timeseries_resp = timeseries_resp.replace('%', '', regex=True)
    timeseries_resp['respchartvalue'] = pd.to_numeric(timeseries_resp['respchartvalue'], errors='coerce')
    timeseries_resp = timeseries_resp.loc[timeseries_resp['respchartvalue'].notnull()]
    timeseries_resp = reconfigure_timeseries(timeseries_resp,
                                             offset_column='respchartoffset',
                                             feature_column='respchartvaluelabel',
                                             test=test)
    timeseries_resp.columns = timeseries_resp.columns.droplevel()

    print('==> Reconfiguring nurse timeseries...')
    # remove non numeric data
    timeseries_nurse['nursingchartvalue'] = pd.to_numeric(timeseries_nurse['nursingchartvalue'], errors='coerce')
    timeseries_nurse = timeseries_nurse.loc[timeseries_nurse['nursingchartvalue'].notnull()]
    timeseries_nurse = reconfigure_timeseries(timeseries_nurse,
                                              offset_column='nursingchartoffset',
                                              feature_column='nursingchartcelltypevallabel',
                                              test=test)
    timeseries_nurse.columns = timeseries_nurse.columns.droplevel()

    print('==> Reconfiguring aperiodic timeseries...')
    timeseries_aperiodic = reconfigure_timeseries(timeseries_aperiodic,
                                                  offset_column='observationoffset',
                                                  test=test)

    print('==> Reconfiguring periodic timeseries...')
    timeseries_periodic = reconfigure_timeseries(timeseries_periodic,
                                                 offset_column='observationoffset',
                                                 test=test)

    patients = timeseries_periodic.index.unique(level=0)

    size = 4000
    gen_chunks = gen_patient_chunk(patients, size=size)
    i = size
    header = True  # for the first chunk include the header in the csv file

    print('==> Starting main processing loop...')

    for patient_chunk in gen_chunks:

        merged = timeseries_lab.loc[patient_chunk].append(timeseries_resp.loc[patient_chunk], sort=False)
        merged = merged.append(timeseries_nurse.loc[patient_chunk], sort=False)
        merged = merged.append(timeseries_periodic.loc[patient_chunk], sort=False)
        merged = merged.append(timeseries_aperiodic.loc[patient_chunk], sort=True)

        if i == size:  # fixed from first run
            # all if not all are not normally distributed
            quantiles = merged.quantile([0.05, 0.95])
            maxs = quantiles.loc[0.95]
            mins = quantiles.loc[0.05]

        merged = 2 * (merged - mins) / (maxs - mins) - 1

        # we then need to make sure that ridiculous outliers are clipped to something sensible
        merged.clip(lower=-4, upper=4, inplace=True)  # room for +- 3 on each side, as variables are scaled roughly between 0 and 1

        resample_and_mask(merged, eICU_path, header, mask_decay=True, decay_rate=4/3, test=test, verbose=False)
        print('==> Processed ' + str(i) + ' patients...')
        i += size
        header = False

    return
def add_time_of_day(processed_timeseries, flat_features):

    print('==> Adding time of day features...')
    processed_timeseries = processed_timeseries.join(flat_features[['hour']], how='inner', on='patient')
    processed_timeseries['hour'] = processed_timeseries['time'] + processed_timeseries['hour']
    hour_list = np.linspace(0, 1, 24)  # make sure it's still scaled well
    processed_timeseries['hour'] = processed_timeseries['hour'].apply(lambda x: hour_list[x%24 - 24])
    return processed_timeseries

def further_processing(eICU_path, test=False):

    if test:
        processed_timeseries = pd.read_csv(eICU_path + 'preprocessed_timeseries.csv', nrows=999999)
    else:
        processed_timeseries = pd.read_csv(eICU_path + 'preprocessed_timeseries.csv')
    processed_timeseries.rename(columns={'Unnamed: 1': 'time'}, inplace=True)
    processed_timeseries.set_index('patient', inplace=True)
    flat_features = pd.read_csv(eICU_path + 'flat_features.csv')
    flat_features.rename(columns={'patientunitstayid': 'patient'}, inplace=True)
    processed_timeseries.sort_values(['patient', 'time'], inplace=True)
    flat_features.set_index('patient', inplace=True)

    processed_timeseries = add_time_of_day(processed_timeseries, flat_features)

    if test is False:
        print('==> Saving finalised preprocessed timeseries...')
        # this will replace old one that was updated earlier in the script
        processed_timeseries.to_csv(eICU_path + 'preprocessed_timeseries.csv')

    return

def timeseries_main(eICU_path, test=False):
    # make sure the preprocessed_timeseries.csv file is not there because the first section of this script appends to it
    if test is False:
        print('==> Removing the preprocessed_timeseries.csv file if it exists...')
        try:
            os.remove(eICU_path + 'preprocessed_timeseries.csv')
        except FileNotFoundError:
            pass
    gen_timeseries_file(eICU_path, test)
    further_processing(eICU_path, test)
    return

In [ ]:
#helper functions that process diagnoses data
def add_codes(splits, codes_dict, words_dict, count):
    codes = list()
    levels = len(splits)  # the max number of levels is 6
    if levels >= 1:
        try:
            codes.append(codes_dict[splits[0]][0])
            codes_dict[splits[0]][2] += 1
        except KeyError:
            codes_dict[splits[0]] = [count, {}, 0]
            codes.append(count)
            words_dict[count] = splits[0]
            count += 1
    if levels >= 2:
        try:
            codes.append(codes_dict[splits[0]][1][splits[1]][0])
            codes_dict[splits[0]][1][splits[1]][2] += 1
        except KeyError:
            codes_dict[splits[0]][1][splits[1]] = [count, {}, 0]
            codes.append(count)
            words_dict[count] = splits[0] + '|' + splits[1]
            count += 1
    if levels >= 3:
        try:
            codes.append(codes_dict[splits[0]][1][splits[1]][1][splits[2]][0])
            codes_dict[splits[0]][1][splits[1]][1][splits[2]][2] += 1
        except KeyError:
            codes_dict[splits[0]][1][splits[1]][1][splits[2]] = [count, {}, 0]
            codes.append(count)
            words_dict[count] = splits[0] + '|' + splits[1] + '|' + splits[2]
            count += 1
    if levels >= 4:
        try:
            codes.append(codes_dict[splits[0]][1][splits[1]][1][splits[2]][1][splits[3]][0])
            codes_dict[splits[0]][1][splits[1]][1][splits[2]][1][splits[3]][2] += 1
        except KeyError:
            codes_dict[splits[0]][1][splits[1]][1][splits[2]][1][splits[3]] = [count, {}, 0]
            codes.append(count)
            words_dict[count] = splits[0] + '|' + splits[1] + '|' + splits[2] + '|' + splits[3]
            count += 1
    if levels >= 5:
        try:
            codes.append(codes_dict[splits[0]][1][splits[1]][1][splits[2]][1][splits[3]][1][splits[4]][0])
            codes_dict[splits[0]][1][splits[1]][1][splits[2]][1][splits[3]][1][splits[4]][2] += 1
        except KeyError:
            codes_dict[splits[0]][1][splits[1]][1][splits[2]][1][splits[3]][1][splits[4]] = [count, {}, 0]
            codes.append(count)
            words_dict[count] = splits[0] + '|' + splits[1] + '|' + splits[2] + '|' + splits[3] + '|' + splits[4]
            count += 1
    if levels is 6:
        try:
            codes.append(codes_dict[splits[0]][1][splits[1]][1][splits[2]][1][splits[3]][1][splits[4]][1][splits[5]][0])
            codes_dict[splits[0]][1][splits[1]][1][splits[2]][1][splits[3]][1][splits[4]][1][splits[5]][2] += 1
        except KeyError:
            codes_dict[splits[0]][1][splits[1]][1][splits[2]][1][splits[3]][1][splits[4]][1][splits[5]] = [count, {}, 0]
            codes.append(count)
            words_dict[count] = splits[0] + '|' + splits[1] + '|' + splits[2] + '|' + splits[3] + '|' + splits[4] + '|' + splits[5]
            count += 1
    return codes, count


def get_mapping_dict(unique_diagnoses):

    # a lot of the notes strings look the same, so we will not propagate beyond Organ Systems for this:
    main_diagnoses = [a for a in unique_diagnoses if not (a.startswith('notes') or a.startswith('admission'))]
    adm_diagnoses = [a for a in unique_diagnoses if a.startswith('admission diagnosis')]
    pasthistory_organsystems = [a for a in unique_diagnoses if a.startswith('notes/Progress Notes/Past History/Organ Systems/')]
    pasthistory_comments = [a for a in unique_diagnoses if a.startswith('notes/Progress Notes/Past History/Past History Obtain Options')]

    # sort into alphabetical order to keep the codes roughly together numerically.
    main_diagnoses.sort()
    adm_diagnoses.sort()
    pasthistory_organsystems.sort()
    pasthistory_comments.sort()

    mapping_dict = {}
    codes_dict = {}
    words_dict = {}
    count = 0

    for diagnosis in main_diagnoses:
        splits = diagnosis.split('|')
        codes, count = add_codes(splits, codes_dict, words_dict, count)
        # add all codes relevant to the diagnosisstring
        mapping_dict[diagnosis] = codes

    for diagnosis in adm_diagnoses:
        # take out the things that are common to all of these because it creates unnecessary levels
        shortened = diagnosis.replace('admission diagnosis|', '')
        shortened = shortened.replace('All Diagnosis|', '')
        shortened = shortened.replace('Additional APACHE  Information|', '')
        splits = shortened.split('|')
        codes, count = add_codes(splits, codes_dict, words_dict, count)
        mapping_dict[diagnosis] = codes

    for diagnosis in pasthistory_organsystems:
        # take out the things that are common to all of these because it creates unnecessary levels
        shortened = diagnosis.replace('notes/Progress Notes/Past History/Organ Systems/', '')
        splits = shortened.split('/')  # note different split to main_diagnoses
        codes, count = add_codes(splits, codes_dict, words_dict, count)
        # add all codes relevant to the diagnosisstring
        mapping_dict[diagnosis] = codes

    for diagnosis in pasthistory_comments:
        # take out the things that are common to all of these because it creates unnecessary levels
        shortened = diagnosis.replace('notes/Progress Notes/Past History/Past History Obtain Options/', '')
        splits = shortened.split('/')  # note different split to main_diagnoses
        codes, count = add_codes(splits, codes_dict, words_dict, count)
        # add all codes relevant to the diagnosisstring
        mapping_dict[diagnosis] = codes

    return codes_dict, mapping_dict, count, words_dict

# get rid of anything that is a parent to only one child (index 2 is 1)
def find_pointless_codes(diag_dict):
    pointless_codes = []
    for key, value in diag_dict.items():
        # if there is only one child, then the branch is linear and can be condensed
        if value[2] is 1:
            pointless_codes.append(value[0])
        # get rid of repeat copies where the parent and child are the same title
        for next_key, next_value in value[1].items():
            if key.lower() == next_key.lower():
                pointless_codes.append(next_value[0])
        pointless_codes += find_pointless_codes(value[1])
    return pointless_codes

# get rid of any codes that have a frequency of less than cut_off
def find_rare_codes(cut_off, sparse_df):
    prevalence = sparse_df.sum(axis=0)  # see if you can stop it making pointless extra classes
    rare_codes = prevalence.loc[prevalence <= cut_off].index
    return list(rare_codes)

def add_apache_diag(sparse_df, raw_data, cut_off):
    print('==> Adding admission diagnoses from flat_features...')
    flat = raw_data['flat_features']
    adm_diag = flat[['patientunitstayid', 'apacheadmissiondx']]
    adm_diag.set_index('patientunitstayid', inplace=True)
    adm_diag = pd.get_dummies(adm_diag, columns=['apacheadmissiondx'])

    rare_adm_diag = find_rare_codes(cut_off, adm_diag)
    groupby_dict = {}
    for diag in adm_diag.columns:
        if diag in rare_adm_diag:
            groupby_dict[diag] = 'groupedapacheadmissiondx_' + diag.split(' ', 1)[0].split('/', 1)[0].split(',', 1)[0][18:]
        else:
            groupby_dict[diag] = diag
    adm_diag = adm_diag.groupby(groupby_dict, axis=1).sum()
    rare_adm_diag = find_rare_codes(cut_off, adm_diag)
    adm_diag.drop(columns=rare_adm_diag, inplace=True)
    all_diag = sparse_df.join(adm_diag, how='outer', on='patientunitstayid')
    return all_diag

def diagnoses_main(eICU_path, raw_data, cut_off_prevalence):
    print('==> Analyzing and processing diagnoses data...')
    diagnoses = raw_data['diagnoses']
    diagnoses.set_index('patientunitstayid', inplace=True)

    unique_diagnoses = diagnoses.diagnosisstring.unique()
    codes_dict, mapping_dict, count, words_dict = get_mapping_dict(unique_diagnoses)

    patients = diagnoses.index.unique()
    index_to_patients = dict(enumerate(patients))
    patients_to_index = {v: k for k, v in index_to_patients.items()}

    diagnoses = diagnoses.groupby('patientunitstayid').apply(lambda diag: diag.to_dict(orient='list')['diagnosisstring']).to_dict()
    diagnoses = {patient: [code for diag in list_diag for code in mapping_dict[diag]] for (patient, list_diag) in diagnoses.items()}

    num_patients = len(patients)
    sparse_diagnoses = np.zeros((num_patients, count))
    for patient, codes in diagnoses.items():
        sparse_diagnoses[patients_to_index[patient], codes] = 1

    sparse_df = pd.DataFrame(sparse_diagnoses, index=patients, columns=range(count))
    cut_off = round(cut_off_prevalence * num_patients)
    sparse_df = add_apache_diag(sparse_df, raw_data, cut_off)

    sparse_df.rename(columns=words_dict, inplace=True)
    print('==> Finalizing and saving processed diagnoses data...')
    sparse_df.to_csv(eICU_path + 'preprocessed_diagnoses.csv')

    return

In [ ]:
#Helper function to preprocess flat features
import pandas as pd


def preprocess_flat(flat):

    # make naming consistent with the other tables
    flat.rename(columns={'patientunitstayid': 'patient'}, inplace=True)
    flat.set_index('patient', inplace=True)

    # admission diagnosis is dealt with in diagnoses.py not flat features
    flat.drop(columns=['apacheadmissiondx'], inplace=True)

    # drop apache variables as these aren't available until 24 hours into the stay
    flat.drop(columns=['eyes', 'motor', 'verbal', 'dialysis', 'vent', 'meds', 'intubated', 'bedcount'], inplace=True)

    flat['gender'].replace({'Male': 1, 'Female': 0}, inplace=True)
    flat['teachingstatus'].replace({'t': 1, 'f': 0}, inplace=True)

    cat_features = ['ethnicity', 'unittype', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
                                         'physicianspeciality', 'numbedscategory', 'region']
    # get rid of any really uncommon values
    for f in cat_features:
        too_rare = [value for value, count in flat[f].value_counts().iteritems() if count < 1000]
        flat.loc[flat[f].isin(too_rare), f] = 'misc'

    # convert the categorical features to one-hot
    flat = pd.get_dummies(flat, columns=cat_features)

    # 10 patients have NaN for age; we fill this with the mean value which is 63
    flat['age'].fillna('63', inplace=True)
    # some of the ages are like '> 89' rather than numbers, this needs removing and converting to numbers
    # but we make an extra variable to keep this information
    flat['> 89'] = flat['age'].str.contains('> 89').astype(int)
    flat['age'] = flat['age'].replace('> ', '', regex=True)
    flat['age'] = [float(value) for value in flat.age.values]

    # note that the features imported from the time series have already been normalised
    # standardisation is for features that are probably normally distributed
    features_for_standardisation = 'admissionheight'
    means = flat[features_for_standardisation].mean(axis=0)
    stds = flat[features_for_standardisation].std(axis=0)
    flat[features_for_standardisation] = (flat[features_for_standardisation] - means) / stds

    # probably not normally distributed
    features_for_min_max = ['admissionweight', 'age', 'hour']#, 'bedcount']

    def scale_min_max(flat):
        quantiles = flat.quantile([0.05, 0.95])
        maxs = quantiles.loc[0.95]
        mins = quantiles.loc[0.05]
        return 2 * (flat - mins) / (maxs - mins) - 1

    flat[features_for_min_max] = flat[features_for_min_max].apply(scale_min_max)

    # we then need to make sure that ridiculous outliers are clipped to something sensible
    flat[features_for_standardisation] = flat[features_for_standardisation].clip(lower=-4, upper=4)  # room for +- 3 on each side of the normal range, as variables are scaled roughly between -1 and 1
    flat[features_for_min_max] = flat[features_for_min_max].clip(lower=-4, upper=4)

    # fill in the NaNs
    # these are mainly found in admissionweight and admissionheight,
    # so we create another variable to tell the model when this has been imputed
    flat['nullweight'] = flat['admissionweight'].isnull().astype(int)
    flat['nullheight'] = flat['admissionheight'].isnull().astype(int)
    flat['admissionweight'].fillna(0, inplace=True)
    flat['admissionheight'].fillna(0, inplace=True)
    # there are only 11 missing genders but we might as well set this to 0.5 to tell the model we aren't sure
    flat['gender'].fillna(0.5, inplace=True)
    flat['gender'].replace({'Other': 0.5, 'Unknown': 0.5}, inplace=True)

    return flat

def preprocess_labels(labels):

    # make naming consistent with the other tables
    labels.rename(columns={'patientunitstayid': 'patient'}, inplace=True)
    labels.set_index('patient', inplace=True)

    labels = pd.get_dummies(labels, columns=['unitdischargelocation', 'unitdischargestatus'])

    labels['actualhospitalmortality'].replace({'EXPIRED': 1, 'ALIVE': 0}, inplace=True)

    return labels

def flat_and_labels_main(raw_data, eICU_path):
    print('==> Preprocessing flat features and labels...')
    flat = preprocess_flat(raw_data['flat_features'])
    labels = preprocess_labels(raw_data['labels'])

    # Code to filter out any patients not present in timeseries data
    try:
        with open(eICU_path + 'stays.txt', 'r') as f:
            ts_patients = [int(patient.rstrip()) for patient in f.readlines()]
    except FileNotFoundError:
        ts_patients = pd.read_csv(eICU_path + 'preprocessed_timeseries.csv')
        ts_patients = [x for x in ts_patients.patient.unique()]
        with open(eICU_path + 'stays.txt', 'w') as f:
            for patient in ts_patients:
                f.write("%s\n" % patient)

    flat = flat.loc[ts_patients].copy()
    labels = labels.loc[ts_patients].copy()

    print('==> Saving finalised preprocessed labels and flat features...')
    flat.to_csv(eICU_path + 'preprocessed_flat.csv')
    labels.to_csv(eICU_path + 'preprocessed_labels.csv')



In [ ]:
#Splitting datasets into training and test
def create_folder(parent_path, folder):
    if not parent_path.endswith('/'):
        parent_path += '/'
    folder_path = parent_path + folder
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    return folder_path

def shuffle_stays(stays, seed=9):
    return shuffle(stays, random_state=seed)

def process_table(table_name, table, stays, folder_path):
    table = table.loc[stays].copy()
    table.to_csv('{}/{}.csv'.format(folder_path, table_name))
    return

def split_train_test(path, is_test=True, seed=9, cleanup=True, MIMIC=False):

    labels = pd.read_csv(path + 'preprocessed_labels.csv')
    labels.set_index('patient', inplace=True)
    # we split by unique patient identifier to make sure there are no patients
    # that cross into both the train and the test sets
    patients = labels.uniquepid.unique()

    train, test = train_test_split(patients, test_size=0.15, random_state=seed)
    train, val = train_test_split(train, test_size=0.15/0.85, random_state=seed)

    print('==> Loading data for splitting...')
    if is_test:
        timeseries = pd.read_csv(path + 'preprocessed_timeseries.csv', nrows=999999)
    else:
        timeseries = pd.read_csv(path + 'preprocessed_timeseries.csv')
    timeseries.set_index('patient', inplace=True)
    if not MIMIC:
        diagnoses = pd.read_csv(path + 'preprocessed_diagnoses.csv')
        diagnoses.set_index('patient', inplace=True)
    flat_features = pd.read_csv(path + 'preprocessed_flat.csv')
    flat_features.set_index('patient', inplace=True)

    # delete the source files, as they won't be needed anymore
    if is_test is False and cleanup:
        print('==> Removing the unsorted data...')
        os.remove(path + 'preprocessed_timeseries.csv')
        if not MIMIC:
            os.remove(path + 'preprocessed_diagnoses.csv')
        os.remove(path + 'preprocessed_labels.csv')
        os.remove(path + 'preprocessed_flat.csv')

    for partition_name, partition in zip(['train', 'val', 'test'], [train, val, test]):
        print('==> Preparing {} data...'.format(partition_name))
        stays = labels.loc[labels['uniquepid'].isin(partition)].index
        folder_path = create_folder(path, partition_name)
        with open(folder_path + '/stays.txt', 'w') as f:
            for stay in stays:
                f.write("%s\n" % stay)
        stays = shuffle_stays(stays, seed=9)
        if MIMIC:
            for table_name, table in zip(['labels', 'flat', 'timeseries'],
                                         [labels, flat_features, timeseries]):
                process_table(table_name, table, stays, folder_path)
        else:
            for table_name, table in zip(['labels', 'flat', 'diagnoses', 'timeseries'],
                                         [labels, flat_features, diagnoses, timeseries]):
                process_table(table_name, table, stays, folder_path)

    return

In [ ]:
# calculate statistics
def calculate_stats(raw_data):
  # implement this function to calculate the statistics
  # it is encouraged to print out the results
  stats={}
  for key, df in raw_data.items():
    stats[key]={
        'rows':df.shape[0],
        'columns':df.shape[1],
        'descriptive_stats': df.describe()
    }
  return stats

stats_data_train = calculate_stats(raw_data)
print("Statistics for diagnoses:")
print("Number of rows:", stats_data_train['diagnoses']['number_of_rows'])
print("Number of columns:", stats_data_train['diagnoses']['number_of_columns'])
print(stats_data_train['diagnoses']['descriptive_statistics'])

# main function that processes all the raw data
def process_data():
  # implement this function to process the data as you need
  print('==> Removing the stays.txt file if it exists...')
  try:
    os.remove( + 'stays.txt')
  except FileNotFoundError:
    pass
  test = False
  cut_off_prevalence = 0.01  # this would be 1%
  gen_timeseries_file(raw_data_dir, test)
  diagnoses_main(raw_data_dir, raw_data, cut_off_prevalence)
  flat_and_labels_main(raw_data, raw_data_dir)
  split_train_test(raw_data_dir, is_test=False)
  return None

processed_data = process_data(raw_data)

''' you can load the processed data directly
processed_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'
def load_processed_data(raw_data_dir):
  pass

'''

##   Model
The model includes the model definitation which usually is a class, model training, and other necessary parts.
  * Model architecture: There are 3 main layers with sub layers that include Conv1d and Linear layers. ReLu, Sigmoid and Hardtanh are used for non-linearities and manage output scales.
  * Training objectives: The loss function using Mean Squared Log Error and Mean Squared Error combination for regression and Binary Cross Entropy for classification. Adam optimizer is used with a learning rate of 0.001 to handly sparse gradients. For multitask learning a weight alpha is used to balance loss components.
  * Others: The model does not use pretrained weights and ther is no implementation of Monte Caro or other methods of uncertainty analysis.
  * The code of model should have classes of the model, functions of model training, model validation, etc.
  * If your model training is done outside of this notebook, please upload the trained model here and develop a function to load and test it.

In [ ]:
#The below code is defining the TPC Model
import torch
import torch.nn as nn
from torch import cat, exp
import torch.nn.functional as F
from torch.nn.functional import pad
from torch.nn.modules.batchnorm import _BatchNorm


###============== The main defining function of the TPC model is temp_pointwise() on line 403 ==============###


# Mean Squared Logarithmic Error (MSLE) loss
class MSLELoss(nn.Module):
    def __init__(self):
        super(MSLELoss, self).__init__()
        self.squared_error = nn.MSELoss(reduction='none')

    def forward(self, y_hat, y, mask, seq_length, sum_losses=False):
        # the log(predictions) corresponding to no data should be set to 0
        log_y_hat = y_hat.log().where(mask, torch.zeros_like(y))
        # the we set the log(labels) that correspond to no data to be 0 as well
        log_y = y.log().where(mask, torch.zeros_like(y))
        # where there is no data log_y_hat = log_y = 0, so the squared error will be 0 in these places
        loss = self.squared_error(log_y_hat, log_y)
        loss = torch.sum(loss, dim=1)
        if not sum_losses:
            loss = loss / seq_length.clamp(min=1)
        return loss.mean()


# Mean Squared Error (MSE) loss
class MSELoss(nn.Module):
    def __init__(self):
        super(MSELoss, self).__init__()
        self.squared_error = nn.MSELoss(reduction='none')

    def forward(self, y_hat, y, mask, seq_length, sum_losses=False):
        # the predictions corresponding to no data should be set to 0
        y_hat = y_hat.where(mask, torch.zeros_like(y))
        # the we set the labels that correspond to no data to be 0 as well
        y = y.where(mask, torch.zeros_like(y))
        # where there is no data log_y_hat = log_y = 0, so the squared error will be 0 in these places
        loss = self.squared_error(y_hat, y)
        loss = torch.sum(loss, dim=1)
        if not sum_losses:
            loss = loss / seq_length.clamp(min=1)
        return loss.mean()


class MyBatchNorm(_BatchNorm):
    def __init__(self, num_features, eps=1e-5, momentum=0.1, affine=True,
                 track_running_stats=True):
        super(MyBatchNorm, self).__init__(
            num_features, eps, momentum, affine, track_running_stats)

    def forward(self, input):
        self._check_input_dim(input)

        # hack to work around model.eval() issue
        if not self.training:
            self.eval_momentum = 0  # set the momentum to zero when the model is validating

        if self.momentum is None:
            exponential_average_factor = 0.0
        else:
            exponential_average_factor = self.momentum if self.training else self.eval_momentum

        if self.track_running_stats:
            if self.num_batches_tracked is not None:
                self.num_batches_tracked = self.num_batches_tracked + 1
                if self.momentum is None:  # use cumulative moving average
                    exponential_average_factor = 1.0 / float(self.num_batches_tracked)
                else:  # use exponential moving average
                    exponential_average_factor = self.momentum if self.training else self.eval_momentum

        return F.batch_norm(
            input, self.running_mean, self.running_var, self.weight, self.bias,
            training=True, momentum=exponential_average_factor, eps=self.eps)  # set training to True so it calculates the norm of the batch


class MyBatchNorm1d(MyBatchNorm):
    def _check_input_dim(self, input):
        if input.dim() != 2 and input.dim() != 3:
            raise ValueError('expected 2D or 3D input (got {}D input)'.format(input.dim()))


class EmptyModule(nn.Module):
    def forward(self, X):
        return X


class TempPointConv(nn.Module):
    def __init__(self, config, F=None, D=None, no_flat_features=None):

        # The timeseries data will be of dimensions B * (2F + 2) * T where:
        #   B is the batch size
        #   F is the number of features for convolution (N.B. we start with 2F because there are corresponding mask features)
        #   T is the number of timepoints
        #   The other 2 features represent the sequence number and the hour in the day

        # The diagnoses data will be of dimensions B * D where:
        #   D is the number of diagnoses
        # The flat data will be of dimensions B * no_flat_features

        super(TempPointConv, self).__init__()
        self.task = config.task
        self.n_layers = config.n_layers
        self.model_type = config.model_type
        self.share_weights = config.share_weights
        self.diagnosis_size = config.diagnosis_size
        self.main_dropout_rate = config.main_dropout_rate
        self.temp_dropout_rate = config.temp_dropout_rate
        self.kernel_size = config.kernel_size
        self.temp_kernels = config.temp_kernels
        self.point_sizes = config.point_sizes
        self.batchnorm = config.batchnorm
        self.last_linear_size = config.last_linear_size
        self.F = F
        self.D = D
        self.no_flat_features = no_flat_features
        self.no_diag = config.no_diag
        self.no_mask = config.no_mask
        self.no_exp = config.no_exp
        self.no_skip_connections = config.no_skip_connections
        self.alpha = config.alpha
        self.momentum = 0.01 if self.batchnorm == 'low_momentum' else 0.1

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.hardtanh = nn.Hardtanh(min_val=1/48, max_val=100)  # keep the end predictions between half an hour and 100 days
        self.msle_loss = MSLELoss()
        self.mse_loss = MSELoss()
        self.bce_loss = nn.BCELoss()

        self.main_dropout = nn.Dropout(p=self.main_dropout_rate)
        self.temp_dropout = nn.Dropout(p=self.temp_dropout_rate)

        self.remove_none = lambda x: tuple(xi for xi in x if xi is not None)  # removes None items from a tuple
        self.empty_module = EmptyModule()

        if self.batchnorm in ['mybatchnorm', 'pointonly', 'temponly', 'low_momentum']:
            self.batchnormclass = MyBatchNorm1d
        elif self.batchnorm == 'default':
            self.batchnormclass = nn.BatchNorm1d

        # input shape: B * D
        # output shape: B * diagnosis_size
        self.diagnosis_encoder = nn.Linear(in_features=self.D, out_features=self.diagnosis_size)

        if self.batchnorm in ['mybatchnorm', 'pointonly', 'low_momentum', 'default']:
            self.bn_diagnosis_encoder = self.batchnormclass(num_features=self.diagnosis_size, momentum=self.momentum)  # input shape: B * diagnosis_size
            self.bn_point_last_los = self.batchnormclass(num_features=self.last_linear_size, momentum=self.momentum)  # input shape: (B * T) * last_linear_size
            self.bn_point_last_mort = self.batchnormclass(num_features=self.last_linear_size, momentum=self.momentum)  # input shape: (B * T) * last_linear_size
        else:
            self.bn_diagnosis_encoder = self.empty_module
            self.bn_point_last_los = self.empty_module
            self.bn_point_last_mort = self.empty_module

        # input shape: (B * T) * last_linear_size
        # output shape: (B * T) * 1
        self.point_final_los = nn.Linear(in_features=self.last_linear_size, out_features=1)
        self.point_final_mort = nn.Linear(in_features=self.last_linear_size, out_features=1)

        if self.model_type == 'tpc':
            self.init_tpc()
        elif self.model_type == 'temp_only':
            self.init_temp()
        elif self.model_type == 'pointwise_only':
            self.init_pointwise()
        else:
            raise NotImplementedError('Specified model type not supported; supported types include tpc, temp_only and pointwise_only')


    def init_tpc(self):

        # non-module layer attributes
        self.layers = []
        for i in range(self.n_layers):
            dilation = i * (self.kernel_size - 1) if i > 0 else 1  # dilation = 1 for the first layer, after that it captures all the information gathered by previous layers
            temp_k = self.temp_kernels[i]
            point_size = self.point_sizes[i]
            self.update_layer_info(layer=i, temp_k=temp_k, point_size=point_size, dilation=dilation, stride=1)

        # module layer attributes
        self.create_temp_pointwise_layers()

        # input shape: (B * T) * ((F + Zt) * (1 + Y) + diagnosis_size + no_flat_features)
        # output shape: (B * T) * last_linear_size
        input_size = (self.F + self.Zt) * (1 + self.Y) + self.diagnosis_size + self.no_flat_features
        if self.no_diag:
            input_size = input_size - self.diagnosis_size
        if self.no_skip_connections:
            input_size = self.F * self.Y + self.Z + self.diagnosis_size + self.no_flat_features
        self.point_last_los = nn.Linear(in_features=input_size, out_features=self.last_linear_size)
        self.point_last_mort = nn.Linear(in_features=input_size, out_features=self.last_linear_size)

        return


    def init_temp(self):

        # non-module layer attributes
        self.layers = []
        for i in range(self.n_layers):
            dilation = i * (self.kernel_size - 1) if i > 0 else 1  # dilation = 1 for the first layer, after that it captures all the information gathered by previous layers
            temp_k = self.temp_kernels[i]
            self.update_layer_info(layer=i, temp_k=temp_k, dilation=dilation, stride=1)

        # module layer attributes
        self.create_temp_only_layers()

        # input shape: (B * T) * (F * (1 + Y) + diagnosis_size + no_flat_features)
        # output shape: (B * T) * last_linear_size
        input_size = self.F * (1 + self.Y) + self.diagnosis_size + self.no_flat_features
        self.point_last_los = nn.Linear(in_features=input_size, out_features=self.last_linear_size)
        self.point_last_mort = nn.Linear(in_features=input_size, out_features=self.last_linear_size)
        return


    def init_pointwise(self):

        # non-module layer attributes
        self.layers = []
        for i in range(self.n_layers):
            point_size = self.point_sizes[i]
            self.update_layer_info(layer=i, point_size=point_size)

        # module layer attributes
        self.create_pointwise_only_layers()

        # input shape: (B * T) * (Zt + 2F + 2 + no_flat_features + diagnosis_size)
        # output shape: (B * T) * last_linear_size
        if self.no_mask:
            input_size = self.Zt + self.F + 2 + self.no_flat_features + self.diagnosis_size
        else:
            input_size = self.Zt + 2 * self.F + 2 + self.no_flat_features + self.diagnosis_size
        self.point_last_los = nn.Linear(in_features=input_size, out_features=self.last_linear_size)
        self.point_last_mort = nn.Linear(in_features=input_size, out_features=self.last_linear_size)

        return


    def update_layer_info(self, layer=None, temp_k=None, point_size=None, dilation=None, stride=None):

        self.layers.append({})
        if point_size is not None:
            self.layers[layer]['point_size'] = point_size
        if temp_k is not None:
            padding = [(self.kernel_size - 1) * dilation, 0]  # [padding_left, padding_right]
            self.layers[layer]['temp_kernels'] = temp_k
            self.layers[layer]['dilation'] = dilation
            self.layers[layer]['padding'] = padding
            self.layers[layer]['stride'] = stride

        return


    def create_temp_pointwise_layers(self):

        ### Notation used for tracking the tensor shapes ###

        # Z is the number of extra features added by the previous pointwise layer (could be 0 if this is the first layer)
        # Zt is the cumulative number of extra features that have been added by all previous pointwise layers
        # Zt-1 = Zt - Z (cumulative number of extra features minus the most recent pointwise layer)
        # Y is the number of channels in the previous temporal layer (could be 0 if this is the first layer)

        self.layer_modules = nn.ModuleDict()

        self.Y = 0
        self.Z = 0
        self.Zt = 0

        for i in range(self.n_layers):

            temp_in_channels = (self.F + self.Zt) * (1 + self.Y) if i > 0 else 2 * self.F  # (F + Zt) * (Y + 1)
            temp_out_channels = (self.F + self.Zt) * self.layers[i]['temp_kernels']  # (F + Zt) * temp_kernels
            linear_input_dim = (self.F + self.Zt - self.Z) * self.Y + self.Z + 2 * self.F + 2 + self.no_flat_features  # (F + Zt-1) * Y + Z + 2F + 2 + no_flat_features
            linear_output_dim = self.layers[i]['point_size']  # point_size
            # correct if no_mask
            if self.no_mask:
                if i == 0:
                    temp_in_channels = self.F
                linear_input_dim = (self.F + self.Zt - self.Z) * self.Y + self.Z + self.F + 2 + self.no_flat_features  # (F + Zt-1) * Y + Z + F + 2 + no_flat_features

            temp = nn.Conv1d(in_channels=temp_in_channels,  # (F + Zt) * (Y + 1)
                             out_channels=temp_out_channels,  # (F + Zt) * Y
                             kernel_size=self.kernel_size,
                             stride=self.layers[i]['stride'],
                             dilation=self.layers[i]['dilation'],
                             groups=self.F + self.Zt)

            point = nn.Linear(in_features=linear_input_dim, out_features=linear_output_dim)

            # correct if no_skip_connections
            if self.no_skip_connections:
                temp_in_channels = self.F * self.Y if i > 0 else 2 * self.F  # F * Y
                temp_out_channels = self.F * self.layers[i]['temp_kernels']  # F * temp_kernels
                #linear_input_dim = self.F * self.Y + self.Z if i > 0 else 2 * self.F + 2 + self.no_flat_features  # (F * Y) + Z
                linear_input_dim = self.Z if i > 0 else 2 * self.F + 2 + self.no_flat_features  # Z
                temp = nn.Conv1d(in_channels=temp_in_channels,
                                 out_channels=temp_out_channels,
                                 kernel_size=self.kernel_size,
                                 stride=self.layers[i]['stride'],
                                 dilation=self.layers[i]['dilation'],
                                 groups=self.F)

                point = nn.Linear(in_features=linear_input_dim, out_features=linear_output_dim)

            if self.batchnorm in ['default', 'mybatchnorm', 'low_momentum']:
                bn_temp = self.batchnormclass(num_features=temp_out_channels, momentum=self.momentum)
                bn_point = self.batchnormclass(num_features=linear_output_dim, momentum=self.momentum)
            elif self.batchnorm == 'temponly':
                bn_temp = self.batchnormclass(num_features=temp_out_channels)
                bn_point = self.empty_module
            elif self.batchnorm == 'pointonly':
                bn_temp = self.empty_module
                bn_point = self.batchnormclass(num_features=linear_output_dim)
            else:
                bn_temp = bn_point = self.empty_module  # linear module; does nothing

            self.layer_modules[str(i)] = nn.ModuleDict({
                'temp': temp,
                'bn_temp': bn_temp,
                'point': point,
                'bn_point': bn_point})

            self.Y = self.layers[i]['temp_kernels']
            self.Z = linear_output_dim
            self.Zt += self.Z

        return


    def create_temp_only_layers(self):

        # Y is the number of channels in the previous temporal layer (could be 0 if this is the first layer)
        self.layer_modules = nn.ModuleDict()
        self.Y = 0

        for i in range(self.n_layers):

            if self.share_weights:
                temp_in_channels = (1 + self.Y) if i > 0 else 2  # (Y + 1)
                temp_out_channels = self.layers[i]['temp_kernels']
                groups = 1
            else:
                temp_in_channels = self.F * (1 + self.Y) if i > 0 else 2 * self.F  # F * (Y + 1)
                temp_out_channels = self.F * self.layers[i]['temp_kernels']  # F * temp_kernels
                groups = self.F

            temp = nn.Conv1d(in_channels=temp_in_channels,
                             out_channels=temp_out_channels,
                             kernel_size=self.kernel_size,
                             stride=self.layers[i]['stride'],
                             dilation=self.layers[i]['dilation'],
                             groups=groups)

            if self.batchnorm in ['default', 'mybatchnorm', 'low_momentum', 'temponly']:
                bn_temp = self.batchnormclass(num_features=temp_out_channels, momentum=self.momentum)
            else:
                bn_temp = self.empty_module  # linear module; does nothing

            self.layer_modules[str(i)] = nn.ModuleDict({
                'temp': temp,
                'bn_temp': bn_temp})

            self.Y = self.layers[i]['temp_kernels']

        return


    def create_pointwise_only_layers(self):

        # Zt is the cumulative number of extra features that have been added by previous pointwise layers
        self.layer_modules = nn.ModuleDict()
        self.Zt = 0

        for i in range(self.n_layers):

            linear_input_dim = self.Zt + 2 * self.F + 2 + self.no_flat_features  # Zt + 2F + 2 + no_flat_features
            linear_output_dim = self.layers[i]['point_size']  # point_size

            if self.no_mask:
                linear_input_dim = self.Zt + self.F + 2 + self.no_flat_features  # Zt + 2F + 2 + no_flat_features

            point = nn.Linear(in_features=linear_input_dim, out_features=linear_output_dim)

            if self.batchnorm in ['default', 'mybatchnorm', 'low_momentum', 'pointonly']:
                bn_point = self.batchnormclass(num_features=linear_output_dim, momentum=self.momentum)
            else:
                bn_point = self.empty_module  # linear module; does nothing

            self.layer_modules[str(i)] = nn.ModuleDict({
                'point': point,
                'bn_point': bn_point})

            self.Zt += linear_output_dim

        return


    # This is really where the crux of TPC is defined. This function defines one TPC layer, as in Figure 3 in the paper:
    # https://arxiv.org/pdf/2007.09483.pdf
    def temp_pointwise(self, B=None, T=None, X=None, repeat_flat=None, X_orig=None, temp=None, bn_temp=None, point=None,
                       bn_point=None, temp_kernels=None, point_size=None, padding=None, prev_temp=None, prev_point=None,
                       point_skip=None):

        ### Notation used for tracking the tensor shapes ###

        # Z is the number of extra features added by the previous pointwise layer (could be 0 if this is the first layer)
        # Zt is the cumulative number of extra features that have been added by all previous pointwise layers
        # Zt-1 = Zt - Z (cumulative number of extra features minus the most recent pointwise layer)
        # Y is the number of channels in the previous temporal layer (could be 0 if this is the first layer)
        # X shape: B * ((F + Zt) * (Y + 1)) * T; N.B exception in the first layer where there are also mask features, in this case it is B * 2F * T
        # repeat_flat shape: (B * T) * no_flat_features
        # X_orig shape: (B * T) * (2F + 2)
        # prev_temp shape: (B * T) * ((F + Zt-1) * (Y + 1))
        # prev_point shape: (B * T) * Z

        Z = prev_point.shape[1] if prev_point is not None else 0

        X_padded = pad(X, padding, 'constant', 0)  # B * ((F + Zt) * (Y + 1)) * (T + padding)
        X_temp = self.temp_dropout(bn_temp(temp(X_padded)))  # B * ((F + Zt) * temp_kernels) * T

        X_concat = cat(self.remove_none((prev_temp,  # (B * T) * ((F + Zt-1) * Y)
                                         prev_point,  # (B * T) * Z
                                         X_orig,  # (B * T) * (2F + 2)
                                         repeat_flat)),  # (B * T) * no_flat_features
                       dim=1)  # (B * T) * (((F + Zt-1) * Y) + Z + 2F + 2 + no_flat_features)

        point_output = self.main_dropout(bn_point(point(X_concat)))  # (B * T) * point_size

        # point_skip input: B * (F + Zt-1) * T
        # prev_point: B * Z * T
        # point_skip output: B * (F + Zt) * T
        point_skip = cat((point_skip, prev_point.view(B, T, Z).permute(0, 2, 1)), dim=1) if prev_point is not None else point_skip

        temp_skip = cat((point_skip.unsqueeze(2),  # B * (F + Zt) * 1 * T
                         X_temp.view(B, point_skip.shape[1], temp_kernels, T)),  # B * (F + Zt) * temp_kernels * T
                        dim=2)  # B * (F + Zt) * (1 + temp_kernels) * T

        X_point_rep = point_output.view(B, T, point_size, 1).permute(0, 2, 3, 1).repeat(1, 1, (1 + temp_kernels), 1)  # B * point_size * (1 + temp_kernels) * T
        X_combined = self.relu(cat((temp_skip, X_point_rep), dim=1))  # B * (F + Zt) * (1 + temp_kernels) * T
        next_X = X_combined.view(B, (point_skip.shape[1] + point_size) * (1 + temp_kernels), T)  # B * ((F + Zt + point_size) * (1 + temp_kernels)) * T

        temp_output = X_temp.permute(0, 2, 1).contiguous().view(B * T, point_skip.shape[1] * temp_kernels)  # (B * T) * ((F + Zt) * temp_kernels)

        return (temp_output,  # (B * T) * ((F + Zt) * temp_kernels)
                point_output,  # (B * T) * point_size
                next_X,  # B * ((F + Zt) * (1 + temp_kernels)) * T
                point_skip)  # for keeping track of the point skip connections; B * (F + Zt) * T


    def temp(self, B=None, T=None, X=None, X_temp_orig=None, temp=None, bn_temp=None, temp_kernels=None, padding=None):

        ### Notation used for tracking the tensor shapes ###

        # Y is the number of channels in the previous temporal layer (could be 0 if this is the first layer)
        # X shape: B * (F * (Y + 1)) * T; N.B exception in the first layer where there are also mask features, in this case it is B * 2F * T
        # X_temp_orig shape: B * F * T

        X_padded = pad(X, padding, 'constant', 0)  # B * (F * (Y + 1)) * (T + padding)

        if self.share_weights:
            _, C, padded_length = X_padded.shape
            chans = int(C / self.F)
            X_temp = self.temp_dropout(bn_temp(temp(X_padded.view(B * self.F, chans, padded_length)))).view(B, (self.F * temp_kernels), T)  # B * (F * temp_kernels) * T
        else:
            X_temp = self.temp_dropout(bn_temp(temp(X_padded)))  # B * (F * temp_kernels) * T

        temp_skip = self.relu(cat((X_temp_orig.unsqueeze(2),  # B * F * 1 * T
                                   X_temp.view(B, self.F, temp_kernels, T)),  # B * F * temp_kernels * T
                                   dim=2))  # B * F * (1 + temp_kernels) * T

        next_X = temp_skip.view(B, (self.F * (1 + temp_kernels)), T)  # B * (F * (1 + temp_kernels)) * T

        return next_X  # B * (F * temp_kernels) * T


    def point(self, B=None, T=None, X=None, repeat_flat=None, X_orig=None, point=None, bn_point=None, point_skip=None):

        ### Notation used for tracking the tensor shapes ###

        # Z is the number of extra features added by the previous pointwise layer (could be 0 if this is the first layer)
        # Zt is the cumulative number of extra features that have been added by all previous pointwise layers
        # Zt-1 = Zt - Z (cumulative number of extra features minus the most recent pointwise layer)
        # X shape: B * (F + Zt) * T; N.B exception in the first layer where there are also mask features, in this case it is B * 2F * T
        # repeat_flat shape: (B * T) * no_flat_features
        # X_orig shape: (B * T) * (2F + 2)
        # prev_point shape: (B * T) * Z

        X_combined = cat((X, repeat_flat), dim=1)

        X_point = self.main_dropout(bn_point(point(X_combined)))  # (B * T) * point_size

        # point_skip input: B * Zt-1 * T
        # prev_point: B * Z * T
        # point_skip output: B * Zt * T
        point_skip = cat(self.remove_none((point_skip, X_point.view(B, T, -1).permute(0, 2, 1))), dim=1)

        # point_skip: B * Zt * T
        # X_orig: (B * T) * (2F + 2)
        # repeat_flat: (B * T) * no_flat_features
        # next_X: (B * T) * (Zt + 2F + 2 + no_flat_features)
        next_X = self.relu(cat((point_skip.permute(0, 2, 1).contiguous().view(B * T, -1), X_orig), dim=1))

        return (next_X,  # (B * T) * (Zt + 2F + 2 + no_flat_features)
                point_skip)  # for keeping track of the pointwise skip connections; B * Zt * T


    def temp_pointwise_no_skip(self, B=None, T=None, temp=None, bn_temp=None, point=None, bn_point=None, padding=None, prev_temp=None,
                               prev_point=None, temp_kernels=None, X_orig=None, repeat_flat=None):

        ### Temporal component ###

        # Y is the number of channels in the previous temporal layer (could be 0 if this is the first layer)
        # prev_temp shape: B * (F * Y) * T; N.B exception in the first layer where there are also mask features, in this case it is B * 2F * T

        X_padded = pad(prev_temp, padding, 'constant', 0)  # B * (F * Y) * (T + padding)
        temp_output = self.relu(self.temp_dropout(bn_temp(temp(X_padded))))  # B * (F * temp_kernels) * T

        ### Pointwise component ###

        # prev_point shape: (B * T) * ((F * Y) + Z)
        point_output = self.relu(self.main_dropout(bn_point(point(prev_point))))  # (B * T) * point_size

        return (temp_output,  # B * (F * temp_kernels) * T
                point_output)  # (B * T) * point_size


    def forward(self, X, diagnoses, flat, time_before_pred=5):

        # flat is B * no_flat_features
        # diagnoses is B * D
        # X is B * (2F + 2) * T
        # X_mask is B * T
        # (the batch is padded to the longest sequence, the + 2 is the time and the hour which are not for temporal convolution)

        # get rid of the time and hour fields - these shouldn't go through the temporal network
        # and split into features and indicator variables
        X_separated = torch.split(X[:, 1:-1, :], self.F, dim=1)  # tuple ((B * F * T), (B * F * T))

        # prepare repeat arguments and initialise layer loop
        B, _, T = X_separated[0].shape
        if self.model_type in ['pointwise_only', 'tpc']:
            repeat_flat = flat.repeat_interleave(T, dim=0)  # (B * T) * no_flat_features
            if self.no_mask:
                X_orig = cat((X_separated[0],
                              X[:, 0, :].unsqueeze(1),
                              X[:, -1, :].unsqueeze(1)), dim=1).permute(0, 2, 1).contiguous().view(B * T, self.F + 2)  # (B * T) * (F + 2)
            else:
                X_orig = X.permute(0, 2, 1).contiguous().view(B * T, 2 * self.F + 2)  # (B * T) * (2F + 2)
            repeat_args = {'repeat_flat': repeat_flat,
                           'X_orig': X_orig,
                           'B': B,
                           'T': T}
            if self.model_type == 'tpc':
                if self.no_mask:
                    next_X = X_separated[0]
                else:
                    next_X = torch.stack(X_separated, dim=2).reshape(B, 2 * self.F, T)  # B * 2F * T
                point_skip = X_separated[0]  # keeps track of skip connections generated from linear layers; B * F * T
                temp_output = None
                point_output = None
            else:  # pointwise only
                next_X = X_orig
                point_skip = None
        elif self.model_type == 'temp_only':
            next_X = torch.stack(X_separated, dim=2).view(B, 2 * self.F, T)  # B * 2F * T
            X_temp_orig = X_separated[0]  # skip connections for temp only model
            repeat_args = {'X_temp_orig': X_temp_orig,
                           'B': B,
                           'T': T}

        if self.no_skip_connections:
            temp_output = next_X
            point_output = cat((X_orig,  # (B * T) * (2F + 2)
                                repeat_flat),  # (B * T) * no_flat_features
                               dim=1)  # (B * T) * (2F + 2 + no_flat_features)
            self.layer1 = True

        for i in range(self.n_layers):
            kwargs = dict(self.layer_modules[str(i)], **repeat_args)
            if self.model_type == 'tpc':
                if self.no_skip_connections:
                    temp_output, point_output = self.temp_pointwise_no_skip(prev_point=point_output, prev_temp=temp_output,
                                                                            temp_kernels=self.layers[i]['temp_kernels'],
                                                                            padding=self.layers[i]['padding'], **kwargs)

                else:
                    temp_output, point_output, next_X, point_skip = self.temp_pointwise(X=next_X, point_skip=point_skip,
                                                                        prev_temp=temp_output, prev_point=point_output,
                                                                        temp_kernels=self.layers[i]['temp_kernels'],
                                                                        padding=self.layers[i]['padding'],
                                                                        point_size=self.layers[i]['point_size'],
                                                                        **kwargs)
            elif self.model_type == 'temp_only':
                next_X = self.temp(X=next_X, temp_kernels=self.layers[i]['temp_kernels'],
                                   padding=self.layers[i]['padding'], **kwargs)
            elif self.model_type == 'pointwise_only':
                next_X, point_skip = self.point(X=next_X, point_skip=point_skip, **kwargs)

        # tidy up
        if self.model_type == 'pointwise_only':
            next_X = next_X.view(B, T, -1).permute(0, 2, 1)
        elif self.no_skip_connections:
            # combine the final layer
            next_X = cat((point_output,
                          temp_output.permute(0, 2, 1).contiguous().view(B * T, self.F * self.layers[-1]['temp_kernels'])),
                         dim=1)
            next_X = next_X.view(B, T, -1).permute(0, 2, 1)

        # note that we cut off at time_before_pred hours here because the model is only valid from time_before_pred hours onwards
        if self.no_diag:
            combined_features = cat((flat.repeat_interleave(T - time_before_pred, dim=0),  # (B * (T - time_before_pred)) * no_flat_features
                                     next_X[:, :, time_before_pred:].permute(0, 2, 1).contiguous().view(B * (T - time_before_pred), -1)), dim=1)  # (B * (T - time_before_pred)) * (((F + Zt) * (1 + Y)) + no_flat_features) for tpc
        else:
            diagnoses_enc = self.relu(self.main_dropout(self.bn_diagnosis_encoder(self.diagnosis_encoder(diagnoses))))  # B * diagnosis_size
            combined_features = cat((flat.repeat_interleave(T - time_before_pred, dim=0),  # (B * (T - time_before_pred)) * no_flat_features
                                     diagnoses_enc.repeat_interleave(T - time_before_pred, dim=0),  # (B * (T - time_before_pred)) * diagnosis_size
                                     next_X[:, :, time_before_pred:].permute(0, 2, 1).contiguous().view(B * (T - time_before_pred), -1)), dim=1)  # (B * (T - time_before_pred)) * (((F + Zt) * (1 + Y)) + diagnosis_size + no_flat_features) for tpc

        last_point_los = self.relu(self.main_dropout(self.bn_point_last_los(self.point_last_los(combined_features))))
        last_point_mort = self.relu(self.main_dropout(self.bn_point_last_mort(self.point_last_mort(combined_features))))

        if self.no_exp:
            los_predictions = self.hardtanh(self.point_final_los(last_point_los).view(B, T - time_before_pred))  # B * (T - time_before_pred)
        else:
            los_predictions = self.hardtanh(exp(self.point_final_los(last_point_los).view(B, T - time_before_pred)))  # B * (T - time_before_pred)
        mort_predictions = self.sigmoid(self.point_final_mort(last_point_mort).view(B, T - time_before_pred))  # B * (T - time_before_pred)

        return los_predictions, mort_predictions


    def temp_pointwise_no_skip_old(self, B=None, T=None, temp=None, bn_temp=None, point=None, bn_point=None, padding=None, prev_temp=None,
                               prev_point=None, temp_kernels=None, X_orig=None, repeat_flat=None):

        ### Temporal component ###

        # Y is the number of channels in the previous temporal layer (could be 0 if this is the first layer)
        # prev_temp shape: B * (F * Y) * T; N.B exception in the first layer where there are also mask features, in this case it is B * 2F * T

        X_padded = pad(prev_temp, padding, 'constant', 0)  # B * (F * Y) * (T + padding)
        temp_output = self.relu(self.temp_dropout(bn_temp(temp(X_padded))))  # B * (F * temp_kernels) * T

        ### Pointwise component ###

        # prev_point shape: (B * T) * ((F * Y) + Z)

        # if this is not layer 1:
        if self.layer1:
            X_concat = prev_point
            self.layer1 = False
        else:
            X_concat = cat((prev_point,
                            prev_temp.permute(0, 2, 1).contiguous().view(B * T, self.F * temp_kernels)),
                           dim=1)

        point_output = self.relu(self.main_dropout(bn_point(point(X_concat))))  # (B * T) * point_size

        return (temp_output,  # B * (F * temp_kernels) * T
                point_output)  # (B * T) * point_size


    def loss(self, y_hat_los, y_hat_mort, y_los, y_mort, mask, seq_lengths, device, sum_losses, loss_type):
        # mort loss
        if self.task == 'mortality':
            loss = self.bce_loss(y_hat_mort, y_mort) * self.alpha
        # los loss
        else:
            bool_type = torch.cuda.BoolTensor if device == torch.device('cuda') else torch.BoolTensor
            if loss_type == 'msle':
                los_loss = self.msle_loss(y_hat_los, y_los, mask.type(bool_type), seq_lengths, sum_losses)
            elif loss_type == 'mse':
                los_loss = self.mse_loss(y_hat_los, y_los, mask.type(bool_type), seq_lengths, sum_losses)
            if self.task == 'LoS':
                loss = los_loss
            # multitask loss
            if self.task == 'multitask':
                loss = los_loss + self.bce_loss(y_hat_mort, y_mort) * self.alpha
        return loss

##   Training
Computational requirements:
The model needs a GPU with high memory bandwidth for faster processing and training. Tools like Conda or Docker can help with dependencies for dev enviorments.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class EICUDataset(Dataset):
    def __init__(self, root_dir, mode='train'):
        """
        root_dir (string): Directory with all the data files.
        mode (string): One of 'train', 'val', or 'test' to specify the subset of data.
        """
        self.flat = pd.read_csv(f"{root_dir}/{mode}/flat.csv")
        self.labels = pd.read_csv(f"{root_dir}/{mode}/labels.csv")

    def __len__(self):
        return len(self.flat)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Assuming your labels.csv has a binary classification target
        labels = self.labels.iloc[idx, 1]  # Modify the index if necessary
        features = self.flat.iloc[idx, 1:].values  # Assuming the first column is not a feature

        # Convert to tensor
        features = torch.from_numpy(features).float()
        labels = torch.tensor(labels).float()

        return features, labels

train_dataset = EICUDataset(raw_data_dir, mode='train')
val_dataset = EICUDataset(raw_data_dir, mode='val')

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

config = {
    'task': 'LoS',
    'n_layers': 3,
    'model_type': 'tpc',
    'share_weights': False,
    'diagnosis_size': 100,
    'main_dropout_rate': 0.1,
    'temp_dropout_rate': 0.1,
    'kernel_size': 3,
    'temp_kernels': [10, 20, 30],  # Example sizes
    'point_sizes': [100, 100, 100],
    'batchnorm': 'default',
    'last_linear_size': 50,
    'F': 50,  # Feature count
    'D': 10,  # Diagnosis vector size
    'no_flat_features': 15,
    'no_diag': False,
    'no_mask': False,
    'no_exp': False,
    'no_skip_connections': False,
    'alpha': 0.5,  # Example for multitask weighting
    'momentum': 0.1
}
model = TempPointConv(config=config)
loss_func = MSLELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

def train_model_one_iter(model, loss_func, optimizer, train_loader, device):
  model.train()
  total_loss = 0
  total_samples = 0
  for batch_idx, batch in enumerate(train_loader):
      if batch_idx > (config.no_train_batches // (100 / config.percentage_data)):
          break

      if config.dataset == 'MIMIC':
          padded, mask, flat, los_labels, mort_labels, seq_lengths = batch
          diagnoses = None
      else:
          padded, mask, diagnoses, flat, los_labels, mort_labels, seq_lengths = batch

      padded, mask, flat = padded.to(device), mask.to(device), flat.to(device)
      if diagnoses is not None:
          diagnoses = diagnoses.to(device)
      los_labels, mort_labels = los_labels.to(device), mort_labels.to(device)

      optimizer.zero_grad()
      y_hat_los, y_hat_mort = model(padded, diagnoses, flat)
      loss = model.loss(y_hat_los, y_hat_mort, los_labels, mort_labels, mask, seq_lengths, device, config.sum_losses, config.loss_type)
      loss.backward()
      optimizer.step()

      total_loss.append(loss.item() * padded.size(0))
      total_samples += padded.size(0)

      if config.intermediate_reporting and batch_idx % config.log_interval == 0 and batch_idx != 0:
          mean_loss = sum(total_loss[-config.log_interval:]) / min(config.log_interval, len(total_loss))
          print(f'Intermediate Training Loss after {batch_idx} batches: {mean_loss:.4f}')

  mean_train_loss = sum(total_loss) / total_samples
  return mean_train_loss


def validate_model(model, valid_loader, device):
  model.eval()
  val_losses = []
  predictions, actuals = [], []
  with torch.no_grad():
      for features, labels in valid_loader:
          features, labels = features.to(device), labels.to(device)
          outputs = model(features)
          loss = loss_func(outputs, labels)
          val_losses.append(loss.item() * features.size(0))
          predictions.extend(outputs.cpu().numpy())
          actuals.extend(labels.cpu().numpy())

  val_loss = np.sum(val_losses) / np.sum([x.size(0) for _, x in valid_loader])
  metrics = print_metrics_regression(np.array(actuals), np.array(predictions))
  return val_loss, metrics


##   Evaluation
Metrics Description
* Mean Absolute Percentage Error (MAPE): Measures the average magnitude of the errors in a set of predictions
* Mean Squared Logarithmic Error (MSLE): Similar to Mean Squared Error, but it first takes the logarithm of the predictions and actual values, which can dampen the effect of large outliers.
* Mean Absolute Deviation (MAD): Represents the average absolute difference between the actual and predicted values.
* Mean Squared Error (MSE): Represents the average squared difference between the estimated values and the actual value.
* R² Score: Provides an indication of goodness of fit and measures how well unseen samples are likely to be predicted by the model, through the proportion of variance explained.
* Cohen’s Kappa: Measures the agreement between the predicted and actual categorizations.

In [ ]:
#These are metrics that will be used to evaluation function
from sklearn import metrics

class CustomBins:
    inf = 1e18
    bins = [(-inf, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 14), (14, +inf)]
    nbins = len(bins)

def get_bin_custom(x, nbins, one_hot=False):
    for i in range(nbins):
        a = CustomBins.bins[i][0]
        b = CustomBins.bins[i][1]
        if a <= x < b:
            if one_hot:
                onehot = np.zeros((CustomBins.nbins,))
                onehot[i] = 1
                return onehot
            return i
    return None

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(4/24, y_true))) * 100  # this stops the mape being a stupidly large value when y_true happens to be very small

def mean_squared_logarithmic_error(y_true, y_pred):
    return np.mean(np.square(np.log(y_true/y_pred)))

def print_metrics_regression(y_true, predictions, verbose=1, elog=None):
    print('==> Length of Stay:')
    y_true_bins = [get_bin_custom(x, CustomBins.nbins) for x in y_true]
    prediction_bins = [get_bin_custom(x, CustomBins.nbins) for x in predictions]
    cf = metrics.confusion_matrix(y_true_bins, prediction_bins)
    if elog is not None:
        elog.print('Custom bins confusion matrix:')
        elog.print(cf)
    elif verbose:
        print('Custom bins confusion matrix:')
        print(cf)

    kappa = metrics.cohen_kappa_score(y_true_bins, prediction_bins, weights='linear')
    mad = metrics.mean_absolute_error(y_true, predictions)
    mse = metrics.mean_squared_error(y_true, predictions)
    mape = mean_absolute_percentage_error(y_true, predictions)
    msle = mean_squared_logarithmic_error(y_true, predictions)
    r2 = metrics.r2_score(y_true, predictions)

    if verbose:
        print('Mean absolute deviation (MAD) = {}'.format(mad))
        print('Mean squared error (MSE) = {}'.format(mse))
        print('Mean absolute percentage error (MAPE) = {}'.format(mape))
        print('Mean squared logarithmic error (MSLE) = {}'.format(msle))
        print('R^2 Score = {}'.format(r2))
        print('Cohen kappa score = {}'.format(kappa))

    return [mad, mse, mape, msle, r2, kappa]

# def print_metrics_mortality(y_true, prediction_probs, verbose=1, elog=None):
#     print('==> Mortality:')
#     prediction_probs = np.array(prediction_probs)
#     prediction_probs = np.transpose(np.append([1 - prediction_probs], [prediction_probs], axis=0))
#     predictions = prediction_probs.argmax(axis=1)
#     cf = metrics.confusion_matrix(y_true, predictions, labels=range(2))
#     if elog is not None:
#         elog.print('Confusion matrix:')
#         elog.print(cf)
#     elif verbose:
#         print('Confusion matrix:')
#         print(cf)
#     cf = cf.astype(np.float32)

#     acc = (cf[0][0] + cf[1][1]) / np.sum(cf)
#     prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
#     prec1 = cf[1][1] / (cf[1][1] + cf[0][1])
#     rec0 = cf[0][0] / (cf[0][0] + cf[0][1])
#     rec1 = cf[1][1] / (cf[1][1] + cf[1][0])

#     auroc = metrics.roc_auc_score(y_true, prediction_probs[:, 1])
#     (precisions, recalls, thresholds) = metrics.precision_recall_curve(y_true, prediction_probs[:, 1])
#     auprc = metrics.auc(recalls, precisions)
#     f1macro = metrics.f1_score(y_true, predictions, average='macro')

#     results = {'Accuracy': acc, 'Precision Survived': prec0, 'Precision Died': prec1, 'Recall Survived': rec0,
#                'Recall Died': rec1, 'Area Under the Receiver Operating Characteristic curve (AUROC)': auroc,
#                'Area Under the Precision Recall curve (AUPRC)': auprc, 'F1 score (macro averaged)': f1macro}
#     if verbose:
#         for key in results:
#             print('{} = {}'.format(key, results[key]))

#     return [acc, prec0, prec1, rec0, rec1, auroc, auprc, f1macro]

In [ ]:
num_epoch = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for epoch in range(num_epoch):
    train_loss = train_model_one_iter(model, loss_func, optimizer, train_loader, device)
    val_loss, metrics = validate_model(model, valid_loader, device)
    print(f"Epoch: {epoch}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    print("Validation Metrics:", metrics)

# Results
In this section, you should finish training your model training or loading your trained model. That is a great experiment! You should share the results with others with necessary metrics and figures.

Please test and report results for all experiments that you run with:

*   specific numbers (accuracy, AUC, RMSE, etc)
*   figures (loss shrinkage, outputs from GAN, annotation or label of sample pictures, etc)


Results
* The primary results from our experiments focused on predicting the Length of Stay (LoS) in medical care facilities using a Temporal Pointwise Convolution (TPC) model. The final experiment settings were tuned to optimize performance specifically for the LoS task. Detailed model performance was tracked using various metrics such as Mean Absolute Deviation (MAD), Mean Squared Error (MSE), Mean Absolute Percentage Error (MAPE), Mean Squared Logarithmic Error (MSLE), R^2 Score, and the Cohen Kappa Score. These metrics provide a comprehensive understanding of the model's accuracy and reliability in predicting patient stay durations.

Analyses (have to add charts next)
* Analysis indicated correlations between the model's predictions and actual outcomes, suggesting that the TPC model effectively captures the temporal dynamics and complexities associated with patient stays. The confusion matrix for custom bins, which categorizes stays into predefined duration intervals, further validated the model’s classification ability across different stay lengths.

Plans
* Add hyperparameter tuning more meticulousely so the best parameters are adopted.
* Possibly analyze other predictive metrics like mortality rate that tpc model can capture.
* Test two differnt ablations relating to loss function and input features.
* Compare this model analysis to other transformer and baseline model metrics.

In [ ]:
import matplotlib.pyplot as plt

# metrics to evaluate my model (calculated in eval step)
# Mean absolute deviation (MAD)
# Mean absolute percentage error (MAPE)
# Mean squared error (MSE)
# Mean squared log error (MSLE)
# Coefficient of determination (R2)

# plot figures to better show the results (will add more later)
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
# it is better to save the numbers and figures for your presentation.

## Model comparison

In [ ]:
# compare you model with others
# you don't need to re-run all other experiments, instead, you can directly refer the metrics/numbers in the paper

# Discussion

In this section,you should discuss your work and make future plan. The discussion should address the following questions:
  * Make assessment that the paper is reproducible or not.
  * Explain why it is not reproducible if your results are kind negative.
  * Describe “What was easy” and “What was difficult” during the reproduction.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
  * What will you do in next phase.



In [ ]:
# no code is required for this section
'''
if you want to use an image outside this notebook for explanaition,
you can read and plot it here like the Scope of Reproducibility
'''

# References

1.   Rocheteau, Emma, et al. “Temporal pointwise convolutional networks for length of stay prediction in the Intensive Care Unit.” Proceedings of the Conference on Health, Inference, and Learning, 8 Apr. 2021, https://doi.org/10.1145/3450439.3451860



# Feel free to add new sections